# LMR reconstructions using temperature sensitive proxies

Adapted from a notebook created by Greg Hakim  
Picked up by Katie Brennan  
May 2019  
October 2019 - add in 95th percentile calculations.   
June 2020 - adapted to work for common era reconstructions

Goal: Assimilate temperature sensitive proxy data into last millennium climate model simulations output (from MPI and CCSM4) to reconstruct Arctic sea ice in the Common Era (0-2000 CE). 

In [1]:
import sys,os,copy
#sys.path.append("/Users/hakim/gitwork/LMR_python3")

In [2]:
import sys
import numpy as np
import pickle

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point

from time import time
from spharm import Spharmt, getspecindx, regrid
from netCDF4 import Dataset
from scipy import stats

In [3]:
sys.path.insert(1,'/home/disk/p/mkb22/Documents/si_analysis_kb/instrumental_assimilation_experiments/')
import reanalysis_recons_utils as rrutils

sys.path.insert(1,'/home/disk/kalman2/mkb22/LMR_lite/')
import LMR_lite_utils as LMRlite
import LMR_utils 
import LMR_config

sys.path.insert(1,'/home/disk/p/mkb22/Documents/si_utils_kb/')
import Sice_utils as siutils 

Loading information from datasets.yml
Loading information from grid_def.yml
Loading information from datasets.yml
Loading information from grid_def.yml


In [4]:
import importlib
# #importlib.reload(LMRlite)
# #importlib.reload(LMR_config)
# importlib.reload(LMRlite)
importlib.reload(rrutils)

<module 'reanalysis_recons_utils' from '/home/disk/p/mkb22/Documents/si_analysis_kb/instrumental_assimilation_experiments/reanalysis_recons_utils.py'>

In [5]:
def sub_arctic_plot(fig,ax,VAR1,LAT,LON,TITLE1,MAX1,colorbar=True,extent=True):
    var1, lon1 = add_cyclic_point(VAR1, coord=LON)
    new_lon2d, new_lat2d = np.meshgrid(lon1, LAT)
    if extent is True: 
        ax.set_extent([-150, 140, 50, 90], crs=ccrs.PlateCarree())
    ax.gridlines(linestyle='--')
    ax.add_feature(cfeature.LAND, facecolor=(1, 1, 1))
    cs = ax.pcolormesh(new_lon2d, new_lat2d, var1, 
                       vmin=-MAX1, vmax=MAX1, cmap=plt.cm.RdBu_r, 
                       transform=ccrs.PlateCarree())
    ax.coastlines(resolution='110m', linewidth=0.5)
    if colorbar is True:
        plt.colorbar(cs, ax=ax)
    ax.set_title(TITLE1)

In [6]:
proj = dict(projection=ccrs.Stereographic(central_latitude=90,
                                          central_longitude=-45,
                                          true_scale_latitude=0.1))

### User parameters: 

In [8]:
recon_start = 1500
recon_end = 1990

# inflate the sea ice variable here (can inflate whole state here too)
inflate = 1.8
inf_name = '1_8'
# inflate = 1.0
# inf_name = '1'

prior_name = 'mpi'

cfile = '/home/disk/kalman2/mkb22/LMR_lite/configs/config_ccsm4_brennan2020.yml'
#cfile = '/home/disk/kalman2/mkb22/LMR_lite/configs/config_ccsm4_fixedprox.yml'
#cfile = '/home/disk/kalman2/mkb22/LMR_lite/configs/config_production2.yml'

loc_list = [15000]

iteration = ['a']

savename = []
proxies = 'pages2kv2'
#proxies = 'fullLMRdbv0_3'

for t,it in enumerate(iteration):

    savedir = ('/home/disk/p/mkb22/nobackup/LMR_output/common_era_experiments/experiments/ccsm4/')
        
#    savename = ['sic_'+prior_name+'_annual_recon_1850_2018_cru_R0_4_10deg_inf'+inf_name+'_loc'+str(loc_list[0])+it+'.pkl']#,
#  #               'sic_'+prior_name+'_annual_recon_1900_1940_cru_R0_4_10deg_inf'+inf_name+'_loc'+str(loc_list[1])+it+'.pkl']
    if it is 0: 
        savename = ['sic_'+prior_name+'_anrecon_'+str(recon_start)+'_'+
                    str(recon_end)+'_'+proxies+'_inf'+inf_name+'_loc'+str(loc_list[0])+'_'+it+'.pkl']
    else: 
        savename.append('sic_'+prior_name+'_anrecon_'+str(recon_start)+'_'+
                        str(recon_end)+'_'+proxies+'_inf'+inf_name+'_loc'+str(loc_list[0])+'_'+it+'.pkl')

In [9]:
savename

['sic_mpi_anrecon_1500_1990_pages2kv2_inf1_8_loc15000_a.pkl']

In [12]:
print('Starting experiment with inflation = '+str(inflate)+', '+inf_name)

print('loading configuration...')

cfg = LMRlite.load_config(cfile)

Starting experiment with inflation = 1.8, 1_8
loading configuration...
Checking configuration ... 
OK!
Checking configuration ... 
OK!
Checking configuration ... 
OK!
Checking configuration ... 
OK!
Checking configuration ... 
OK!
Checking configuration ... 
OK!
Checking configuration ... 
OK!
Checking configuration ... 
OK!
Checking configuration ... 
OK!
Checking configuration ... 
OK!


In [38]:
print('loading prior')
X, Xb_one = LMRlite.load_prior(cfg)
Xbp = Xb_one - Xb_one.mean(axis=1,keepdims=True)

# use this for labeling graphics and output files
prior_id_string = X.prior_datadir.split('/')[-1]
print('prior string label: ',prior_id_string)

# check if config is set to regrid the prior
if cfg.prior.regrid_method:
    print('regridding prior...')
    # this function over-writes X, even if return is given a different name
    [X_regrid,Xb_one_new] = LMRlite.prior_regrid(cfg,X,Xb_one,verbose=True)
else:
    X_regrid.trunc_state_info = X.full_state_info

loading prior
('Reading file: ', '/home/disk/kalman3/rtardif/LMR/data/model/ccsm4_last_millenium/tas_sfc_Amon_CCSM4_past1000_085001-185012.nc')
(12012, 192, 288)
('indlat=', 0, ' indlon=', 1)
Anomalies provided as the prior: Removing the temporal mean (for every gridpoint)...
('tas', ': Global(monthly): mean=', 8.072375e-07, ' , std-dev=', 1.8899411)
('Averaging over month sequence:', [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
('tas', ': Global(time-averaged): mean=', 4.4424884352419226e-08, ' , std-dev=', 0.8317386411161235)
('Reading file: ', '/home/disk/kalman3/rtardif/LMR/data/model/ccsm4_last_millenium/sic_sfc_OImon_CCSM4_past1000_085001-185012.nc')
(12012, 180, 360)
('indlat=', 0, ' indlon=', 1)
Full field provided as the prior
('sic', ': Global(monthly): mean=', 13.445376, ' , std-dev=', 32.317898)
('Averaging over month sequence:', [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
('sic', ': Global(time-averaged): mean=', 13.445537533002385, ' , std-dev=', 31.06984518565762)
 
State vecto

In [14]:
# make a grid object for the prior
grid = LMRlite.Grid(X_regrid)

# locate 2m air temperature in Xb_one and make a new array
tas_pos = X_regrid.trunc_state_info['tas_sfc_Amon']['pos']
tas = Xb_one_new[tas_pos[0]:tas_pos[1]+1,:]

# fix using file system softlink for ccsm4 filename...
sic_pos = X_regrid.trunc_state_info['sic_sfc_OImon']['pos']
print('assigning '+prior_id_string+ ' sea ice ...')

sic = Xb_one_new[sic_pos[0]:sic_pos[1]+1,:]

assigning ccsm4_last_millenium sea ice ...


In [ ]:
# lalo_dir = '/home/disk/p/mkb22/nobackup/LMR_output/common_era_experiments/experiments/'
# lalo_savename = '20CR_grid.pkl'
# pickle.dump(grid,open(lalo_dir+lalo_savename, "wb"))

In [15]:
# load proxies
prox_manager = LMRlite.load_proxies(cfg)

                                 Bivalve_d18O :     1
               Corals and Sclerosponges_Rates :     2
                Corals and Sclerosponges_SrCa :     7
                Corals and Sclerosponges_d18O :    25
                        Ice Cores_MeltFeature :     1
                               Ice Cores_d18O :    24
                                 Ice Cores_dD :     3
                              Lake Cores_Misc :     0
                             Lake Cores_Varve :     4
                       Tree Rings_WidthPages2 :   253
                       Tree Rings_WoodDensity :    45
                                        TOTAL :   365
-----------------------------------------------------
completed in 2.1848809719085693 seconds
-----------------------------------------------------


In [ ]:
prox_assim_info = {'Tree Rings_WidthPages2':{'lat':[],'lon':[],'color':'g', 'label':'Tree'},
                   'Tree Rings_WidthBreit':{'lat':[],'lon':[],'color':'g', 'label':'Tree'},
                   'Tree Rings_WoodDensity':{'lat':[],'lon':[],'color':'g', 'label':'Tree'}, 
                   'Corals and Sclerosponges_d18O':{'lat':[],'lon':[],'color':'coral', 'label':'Coral'}, 
                   'Corals and Sclerosponges_SrCa':{'lat':[],'lon':[],'color':'coral', 'label':'Coral'}, 
                   'Corals and Sclerosponges_Rates':{'lat':[],'lon':[],'color':'coral', 'label':'Coral'},
                   'Ice Cores_d18O':{'lat':[],'lon':[],'color':'darkturquoise', 'label':'Ice Core'},
                   'Ice Cores_dD':{'lat':[],'lon':[],'color':'darkturquoise', 'label':'Ice Core'}, 
                   'Ice Cores_MeltFeature':{'lat':[],'lon':[],'color':'darkturquoise', 'label':'Ice Core'},
                   'Ice Cores_Accumulation':{'lat':[],'lon':[],'color':'darkturquoise', 'label':'Ice Core'},
                   'Lake Cores_Varve':{'lat':[],'lon':[],'color':'royalblue', 'label':'Lake Core'}, 
                   'Lake Cores_Misc':{'lat':[],'lon':[],'color':'royalblue', 'label':'Lake Core'}, 
                   'Bivalve_d18O':{'lat':[],'lon':[],'color':'saddlebrown', 'label':'Bivalve'}}

for proxy_idx, Y in enumerate(prox_manager.sites_assim_proxy_objs()):
    prox_assim_info[Y.type]['lat'] = np.append(prox_assim_info[Y.type]['lat'],Y.lat)
    prox_assim_info[Y.type]['lon'] = np.append(prox_assim_info[Y.type]['lon'],Y.lon)

In [ ]:
#Gather proxy info to save: 
prox_present = []
nprox_assim = 0

for Y in prox_manager.sites_assim_proxy_objs():
    if Y.type not in prox_present:
        prox_present = np.append(prox_present,Y.type)
    nprox_assim = nprox_assim+1

prox_assim_info = {}

for P in prox_present: 
    prox_assim_info[P] = {'lat':[],'lon':[]}

for proxy_idx, Y in enumerate(prox_manager.sites_assim_proxy_objs()):
    prox_assim_info[Y.type]['lat'] = np.append(prox_assim_info[Y.type]['lat'],Y.lat)
    prox_assim_info[Y.type]['lon'] = np.append(prox_assim_info[Y.type]['lon'],Y.lon)

In [ ]:
prox_info_assim = {}

prox_info_assim.keys(list(prox_present))

In [ ]:
# Plot assimilated proxies: Arctic 
handle_list = []
label_list = []

fig,ax = plt.subplots(1,1, figsize=(6, 6), subplot_kw = proj)
sub_arctic_plot(fig,ax,np.zeros((grid.nlat,grid.nlon)),
                grid.lat[:,0],grid.lon[0,:],
                'Assimilated proxy locations (total # = '+str(nprox_assim)+')',
                1, colorbar=False)
for loc in prox_present: 
    ax.scatter([prox_assim_info[loc]['lon']],[prox_assim_info[loc]['lat']],
               color=prox_assim_info[loc]['color'],transform=ccrs.PlateCarree(), 
               label=prox_assim_info[loc]['label'], edgecolors='k',s=50)

handles, labels = ax.get_legend_handles_labels()
for handle, label in zip(handles, labels):
    if label not in label_list:
        handle_list.append(handle)
        label_list.append(label)
ax.legend(handle_list, label_list,bbox_to_anchor=(1.05, 0.8))

#fig.suptitle('SIC Truth vs Reconstruction: 1681-1850', fontsize=14)
#plt.tight_layout(rect=(0,0,1,0.93))

# savedir = '/home/disk/p/mkb22/Documents/si_analysis_kb/common_era_experiments/analysis/figures/sic_ccsm4_anrecon_0_2000_fullLMRdbv1_1_inf2_6_loc15000_a/'
# savename = 'assim_proxies_arc_2090.png'
# plt.savefig(savedir+savename)

In [ ]:
# Plot assimilated proxies: Global 
handle_list = []
label_list = []

fig,ax = plt.subplots(1,1, figsize=(8,6), subplot_kw = dict(projection=ccrs.Robinson()))
sub_arctic_plot(fig,ax,np.zeros((grid.nlat,grid.nlon)),
                grid.lat[:,0],grid.lon[0,:],
                'Assimilated proxy locations (total # = '+str(nprox_assim)+')',1, 
                colorbar=False, extent=False)
for loc in prox_present: 
    ax.scatter([prox_assim_info[loc]['lon']],[prox_assim_info[loc]['lat']],
               color=prox_assim_info[loc]['color'],transform=ccrs.PlateCarree(), 
               label=prox_assim_info[loc]['label'], edgecolors='k',s=50)

handles, labels = ax.get_legend_handles_labels()
for handle, label in zip(handles, labels):
    if label not in label_list:
        handle_list.append(handle)
        label_list.append(label)
ax.legend(handle_list, label_list,bbox_to_anchor=(1.05, 0.8))

# savedir = '/home/disk/p/mkb22/Documents/si_analysis_kb/common_era_experiments/analysis/figures/sic_ccsm4_anrecon_0_2000_fullLMRdbv1_1_inf2_6_loc15000_a/'
# savename = 'assim_proxies_global_2090.png'
# plt.savefig(savedir+savename)

In [ ]:
# All proxies available given config setup: 
type_site_assim = prox_manager.all_ids_by_group
assim_proxy_count = len(prox_manager.all_proxies)
for pkey, plist in sorted(type_site_assim.items()):
    print(('%45s : %5d' % (pkey, len(plist))))
print(('%45s : %5d' % ('TOTAL', assim_proxy_count)))

In [16]:
#--------------------------------------------------------
# start reconstruction experiments here for ALL reference datasets
#--------------------------------------------------------
# set parameters here

# NH surface area in M km^2 from concentration in percentage
nharea = 2*np.pi*(6380**2)/1e8

# inflate the entire state vector
if 2 == 1:
    print('inflating full state vector...')
    xbm = np.mean(Xb_one_new,1)
    xbp = Xb_one_new - xbm[:,None]
    Xb_one_inflate = np.copy(Xb_one_new)
    Xb_one_inflate = np.add(inflate*xbp,xbm[:,None])
else:
    # inflate sea ice only
    print('inflating only sea ice by '+str(inflate))
    xb_sicm = np.mean(Xb_one_new[sic_pos[0]:sic_pos[1]+1,:],1)
    xb_sicp = np.squeeze(Xb_one_new[sic_pos[0]:sic_pos[1]+1,:])-xb_sicm[:,None]
    sic_new = np.add(inflate*xb_sicp,xb_sicm[:,None])
    Xb_one_inflate = np.copy(Xb_one_new)
    Xb_one_inflate[sic_pos[0]:sic_pos[1]+1,:] = sic_new

inflating only sea ice by 1.8


In [17]:
print('loading Ye...')
[Ye_assim, 
Ye_assim_coords] = LMR_utils.load_precalculated_ye_vals_psm_per_proxy(cfg, 
                                                                      prox_manager,
                                                                      'assim',
                                                                      X.prior_sample_indices)

loading Ye...
  Loading file: ccsm4_last_millenium-pr_sfc_Amon-tas_sfc_Amon-anom_bilinear_seasonPSM-T:GISTEMP-PR:GPCC_ref1951-1980_cal1850-2015_LMRdbv1.1.0.npz
  Loading file: ccsm4_last_millenium-tas_sfc_Amon-anom_linear_seasonPSM-GISTEMP_ref1951-1980_cal1850-2015_LMRdbv1.1.0.npz
  Now extracting proxy type-dependent Ye values...
  Completed in  7.268322944641113 secs


In [18]:
# nyear = 1000.0
# vY = []
# for proxy_idx, Y in enumerate(prox_manager.sites_assim_proxy_objs()):
#     try: 
#         vY.append(Y.values[nyear])
#     except KeyError: 
#         continue

In [19]:
# sic_dict_lalo = {}
# sic_ens_full = {}
# tas_dict_lalo = {}
# var_dict = {}
# obs_full = {}
# obs_size = {}
# Ye_full = {}
# sic_ens_dict = {}
# sie_ens_dict = {}

#cfg.core.loc_rad = loc
print('Localization radius: ',cfg.core.loc_rad)

#--------------------------------------------------------------------------------
# Loop over all years available in this reference dataset
recon_years = range(cfg.core.recon_period[0],cfg.core.recon_period[1])
#recon_years = range(1979,2000)
nyears = len(recon_years)

sic_save = []
nobs = []
sic_save_lalo = np.zeros((nyears,grid.nlat,grid.nlon))
tas_save_lalo = np.zeros((nyears,grid.nlat,grid.nlon))
sic_lalo_full = np.zeros((grid.nlat,grid.nlon,grid.nens))
var_save = []
sic_full_ens = []
sie_full_ens = []
sic_ndim = sic_pos[1]-sic_pos[0]+1

begin_time = time()
yk = -1
for yk, target_year in enumerate(recon_years):
    print('working on: '+ str(target_year))
    
    # Do data assimilation
    [vY,vR,vP,vYe,
     vT,vYe_coords] = LMRlite.get_valid_proxies(cfg,prox_manager,target_year,
                                                Ye_assim,Ye_assim_coords, verbose=False)
    nobs = np.append(nobs,len(vY))
    
    #xam,Xap,_ = LMRlite.Kalman_optimal(obs_QC,R_QC,Ye_QC,Xb_one_inflate)
    xam,Xap = LMRlite.Kalman_ESRF(cfg,vY,vR,vYe,
                                  Xb_one_inflate,X=X_regrid,
                                  vYe_coords=vYe_coords,verbose=False)

    tas_lalo = np.reshape(xam[tas_pos[0]:tas_pos[1]+1],[grid.nlat,grid.nlon])
    tas_save_lalo[yk,:,:] = tas_lalo

    # this saves sea-ice area for the entire ensemble
    sic_ens = []
    sie_ens = []
 #   for k in range(grid.nens):
    sic_lalo = np.reshape(xam[sic_pos[0]:sic_pos[1]+1,np.newaxis]+Xap[sic_pos[0]:sic_pos[1]+1,:],
                          [grid.nlat,grid.nlon,grid.nens])
    if 'full' in cfg.prior.state_variables['sic_sfc_OImon']:
        sic_lalo = np.where(sic_lalo<0.0,0.0,sic_lalo)
        sic_lalo = np.where(sic_lalo>100.0,100.0,sic_lalo)

        # Calculate extent: 
        sie_lalo = siutils.calc_sea_ice_extent(sic_lalo,15.0)
    else: 
        sic_lalo = sic_lalo
        sie_lalo = np.zeros(sic_lalo.shape)
            
    for k in range(grid.nens):
        _,nhmic,_ = LMR_utils.global_hemispheric_means(sic_lalo[:,:,k],grid.lat[:, 0])
        _,sie_nhmic,_ = LMR_utils.global_hemispheric_means(sie_lalo[:,:,k],grid.lat[:, 0])
        sic_ens.append(nhmic)
        sie_ens.append(sie_nhmic)

    sic_save_lalo[yk,:,:] = np.nanmean(sic_lalo,axis=2)
    var_save.append(np.var(sic_ens,ddof=1))
    sic_full_ens.append(sic_ens)
    sie_full_ens.append(sie_ens)

    # this saves the gridded concentration field for the entire ensemble
    #Xap_save[0,:,:] = Xap[sic_pos[0]:sic_pos[1]+1]
    #Xap_var = np.var(Xap[sic_pos[0]:sic_pos[1]+1,:],axis=1,ddof=1)
    #Xap_var_save[0,:] = Xap_var

    print('done reconstructing: ',target_year)

elapsed_time = time() - begin_time
print('-----------------------------------------------------')
print('completed in ' + str(elapsed_time) + ' seconds')
print('-----------------------------------------------------')

Localization radius:  15000
working on: 1400
Xb shape: (32889, 200)
X coords shape (32760, 2)
X aug coords shape (32889, 2)
done reconstructing:  1400
working on: 1401
Xb shape: (32890, 200)
X coords shape (32760, 2)
X aug coords shape (32890, 2)


KeyboardInterrupt: 

In [37]:
X_regrid.coords.shape

(32760, 2)

In [21]:
cfg,vY,vR,vYe,Xb_one_inflate,X=X_regrid,vYe_coords=vYe_coords,

1

In [ ]:
sic_recon = {}
sic_recon['sic_lalo'] = sic_save_lalo
sic_recon['tas_lalo'] = tas_save_lalo
sic_recon['sic_ens_var'] = np.mean(var_save)
sic_recon['nobs'] = nobs
sic_recon['sia_ens'] = np.squeeze(np.array(sic_full_ens))*nharea
sic_recon['sie_ens'] = np.squeeze(np.array(sie_full_ens))*nharea
sic_recon['recon_years'] = recon_years
sic_recon['Ye_assim'] = Ye_assim
sic_recon['Ye_assim_coords'] = Ye_assim_coords

#sic_recon['sic_ens_full'] = sic_ens_full
#sic_recon['sic_full_ens'] = sic_full_ens
#sic_recon['obs_full'] = obs_full

print('Saving to: ',savedir+savename[0])
pickle.dump(sic_recon,open(savedir+savename[0], "wb"))

In [ ]:
# LOAD ANNUAL SATELLITE DATA: 
[fet_sia_anom, fet_sie_anom, fet_time] = rrutils.load_annual_satellite_anom(2000)
[fet_sia, fet_sie, fet_time2] = rrutils.load_annual_satellite()

In [ ]:
sie_ens_anom = sic_recon['sie_ens'] - np.nanmean(sic_recon['sie_ens'],axis=0)
sie_ensmn_anom = np.nanmean(sie_ens_anom,axis=1)

sie_97_5 = np.percentile(sie_ens_anom,97.5,axis=1)
sie_2_5 = np.percentile(sie_ens_anom,2.5,axis=1)

[ce_sie,corr_sie,var_sie_sat,var_sie_recon] = rrutils.find_ce_corr(sie_ensmn_anom, fet_sie_anom,fet_time,
                                                         np.array(sic_recon['recon_years']),1979,1999)

In [ ]:
str(cfg.core.recon_period[1])

In [ ]:
plt.figure(figsize=(10,6))

plt.plot(sic_recon['recon_years'],sie_ensmn_anom,label='Ensemble mean',linewidth=2.5)
plt.plot(fet_time,fet_sie_anom,label='Satellite (Fetterer et al. 2017)', color='r',linewidth=2.5)
plt.fill_between(sic_recon['recon_years'],sie_2_5,sie_97_5,alpha=0.25)

plt.ylabel('Sea ice extent anomalies (10$^{6}$ $km^2$)',fontsize=14)
plt.title('Total Arctic Sea Ice Extent', fontsize=14)

plt.legend(fontsize=12)
#plt.ylim(7,13.1,1)
ce_corr_plt = (('(Satellite, DA reconstruction): R$^{2}$ = ' + '{:,.2f}'.format(corr_sie**2))+
               ', CE = '+'{:,.2f}'.format(ce_sie))
plt.gcf().text(0.14, 0.14, ce_corr_plt , fontsize=13,color='k')

#print(ce_sie,corr_sie,var_sie_sat,var_sie_recon)

savedir = ('/home/disk/p/mkb22/Documents/si_analysis_kb/common_era_experiments/'+
           'analysis/figures/sic_ccsm4_anrecon_0_2000_fullLMRdbv1_2_6_inf1_loc15000_a/')
savename = 'tot_sie_v_sat_1979_2017.png'
plt.savefig(savedir+savename)

In [ ]:
sic_anom.max()

In [ ]:
# Plot assimilated proxies: Arctic
recon_years = np.array(sic_recon['recon_years'])

sic_sat_mean = np.nanmean(sic_recon['sic_lalo'],axis=0)
sic_anom = sic_recon['sic_lalo'] - sic_sat_mean

fig,ax = plt.subplots(6,4, figsize=(20,30), subplot_kw = proj)
ax = ax.flatten()

for yr in range(recon_years.shape[0]):
    sub_arctic_plot(fig,ax[yr],sic_anom[yr,:,:],
                    grid.lat[:,0],grid.lon[0,:],str(recon_years[yr]),20, colorbar=False)

fig.suptitle('SIC Reconstructed anomalies: 1979-2000', fontsize=20)
plt.tight_layout(rect=(0,0,1,0.97))

# savedir = '/home/disk/p/mkb22/Documents/si_analysis_kb/common_era_experiments/analysis/figures/sic_ccsm4_anrecon_0_2000_pages2kv1_inf2_6_loc15000_a/'
# savename = 'sie_recon_anoms_1979_2017.png'
# plt.savefig(savedir+savename)

## Run this cell with 1000 ens

In [ ]:
colorb = np.

In [ ]:
plt.fill_between(recon_years_all['CRU'],sie_2_5_200ens,sie_97_5_ccsm4_200ens,
                 alpha=0.3, color=plt.cm.Greens(100))
plt.fill_between(recon_years_all['CRU'],sie_2_5_1000ens,sie_97_5_1000ens,
                 alpha=0.3,color=plt.cm.Greens(200))
plt.fill_between(recon_years_all['CRU'],sie_2_5_100ens,sie_97_5_ccsm4_100ens,
                 alpha=0.3,color=plt.cm.Greens(300))

plt.fill_between(recon_years_all['CRU'],sie_2_5_ccsm4_200ens,sie_97_5_ccsm4_200ens,
                 alpha=0.3, color=plt.cm.Blues(100))
plt.fill_between(recon_years_all['CRU'],sie_2_5_ccsm4_1000ens,sie_97_5_1000ens,
                 alpha=0.3,color=plt.cm.Blues(100))
plt.fill_between(recon_years_all['CRU'],sie_2_5_ccsm4_100ens,sie_97_5_ccsm4_100ens,
                 alpha=0.3,color=plt.cm.Blues(100))

plt.plot(recon_years_all['CRU'],sie_100ens,
         label='MPI - 100 ensemble members',color=plt.cm.Greens(100),linewidth=3)
plt.plot(recon_years_all['CRU'],sie_1000ens, 
         color=plt.cm.Greens(200),label='MPI - 1000 ensemble members',linewidth=3)
plt.plot(recon_years_all['CRU'],sie_200ens, 
         color=plt.cm.Greens(300),label='MPI - 200 ensemble members',linewidth=3)

plt.plot(recon_years_all['CRU'],sie_ccsm4_100ens,
         label='CCSM4 - 100 ensemble members',color=plt.cm.Blues(100),linewidth=3)
plt.plot(recon_years_all['CRU'],sie_ccsm4_1000ens, 
         color=plt.cm.Blues(200),label='CCSM4 - 1000 ensemble members',linewidth=3)
plt.plot(recon_years_all['CRU'],sie_ccsm4_200ens, 
         color=plt.cm.Blues(300),label='CCSM4 - 200 ensemble members',linewidth=3)

# plt.plot(recon_years_all['CRU'],np.nanmean(sie_ens_dict[ref_dset],axis=1),label='1000 ensemble members')
# plt.fill_between(recon_years_all['CRU'],sie_2_5,sie_97_5,alpha=0.5)

plt.title('Sea Ice Extent CCSM4 - ensemble spread')
plt.ylabel('sea ice extent (x$10^{6}$ $km^{2}$)')
plt.legend()

In [ ]:
[ce_ccsm4_100_200, corr_ccsm4_100_200,
 var_ref_ccsm4_100_200,var_var_ccsm4_100_200] = rrutils.find_ce_corr(sie_ccsm4_100ens, 
                                                                     sie_ccsm4_200ens,
                                                                     analysis_time['CRU'], 
                                                                     np.arange(1850,2019,1),
                                                                     1850, 2018, detrend=False)
[ce_ccsm4_1000_200, corr_ccsm4_1000_200,
 var_ref_ccsm4_1000_200,var_var_ccsm4_1000_200] = rrutils.find_ce_corr(sie_ccsm4_200ens, 
                                                           sie_ccsm4_1000ens,
                                                           analysis_time['CRU'], 
                                                           np.arange(1850,2020,1), 
                                                           1850, 2018, detrend=False)
[ce_ccsm4_1000_100, corr_ccsm4_1000_100,
 var_ref_ccsm4_1000_100,var_var_ccsm4_1000_100] = rrutils.find_ce_corr(sie_ccsm4_100ens, 
                                                                       sie_ccsm4_1000ens,
                                                                       analysis_time['CRU'],
                                                                       np.arange(1850,2019,1),
                                                                       1850, 2018, detrend=False)

print('CCSM4 100 vs. 200 ens  r = '+str(np.round(corr_ccsm4_100_200,3)))
print('CCSM4 100 vs. 1000 ens r = '+str(np.round(corr_ccsm4_1000_100,3)))
print('CCSM4 200 vs. 1000 ens r = '+str(np.round(corr_ccsm4_1000_200,3)))

print('CCSM4 100 vs. 200 ens  ce = '+str(np.round(ce_ccsm4_100_200,3)))
print('CCSM4 100 vs. 1000 ens ce = '+str(np.round(ce_ccsm4_1000_100,3)))
print('CCSM4 200 vs. 1000 ens ce = '+str(np.round(ce_ccsm4_1000_200,3)))

In [ ]:
[ce_100_200, corr_100_200,
 var_ref_100_200,var_var_100_200] = rrutils.find_ce_corr(sie_100ens, sie_200ens,
                                                     analysis_time['CRU'], np.arange(1850,2019,1), 
                                                     1850, 2018, detrend=False)
[ce_1000_200, corr_1000_200,
 var_ref_1000_200,var_var_1000_200] = rrutils.find_ce_corr(sie_200ens, sie_1000ens,
                                                     analysis_time['CRU'], np.arange(1850,2020,1), 
                                                     1850, 2018, detrend=False)
[ce_1000_100, corr_1000_100,
 var_ref_1000_100,var_var_1000_100] = rrutils.find_ce_corr(sie_100ens, sie_1000ens,
                                                     analysis_time['CRU'], np.arange(1850,2019,1), 
                                                     1850, 2018, detrend=False)

print('MPI 100 vs. 200 ens  r = '+str(np.round(corr_100_200,3)))
print('MPI 100 vs. 1000 ens r = '+str(np.round(corr_1000_100,3)))
print('MPI 200 vs. 1000 ens r = '+str(np.round(corr_1000_200,3)))

print('MPI 100 vs. 200 ens  ce = '+str(np.round(ce_100_200,3)))
print('MPI 100 vs. 1000 ens ce = '+str(np.round(ce_1000_100,3)))
print('MPI 200 vs. 1000 ens ce = '+str(np.round(ce_1000_200,3)))

In [ ]:
[ce_ccsm4_mpi_100_200, corr_ccsm4_mpi_100_200,
 var_ref_ccsm4_mpi_100_200,var_var_ccsm4_mpi_100_200] = rrutils.find_ce_corr(sie_ccsm4_100ens, 
                                                                     sie_ccsm4_200ens,
                                                                     analysis_time['CRU'], 
                                                                     np.arange(1850,2019,1),
                                                                     1850, 2018, detrend=False)
[ce_ccsm4_mpi_1000_200, corr_ccsm4_mpi_1000_200,
 var_ref_ccsm4_mpi1000_200,var_var_ccsm4_mpi1000_200] = rrutils.find_ce_corr(sie_ccsm4_200ens, 
                                                           sie_ccsm4_1000ens,
                                                           analysis_time['CRU'], 
                                                           np.arange(1850,2020,1), 
                                                           1850, 2018, detrend=False)
[ce_ccsm4_mpi_1000_100, corr_ccsm4_mpi1000_100,
 var_ref_ccsm4_mpi_1000_100,var_var_ccsm4_mpi_1000_100] = rrutils.find_ce_corr(sie_ccsm4_100ens, 
                                                                       sie_ccsm4_1000ens,
                                                                       analysis_time['CRU'],
                                                                       np.arange(1850,2019,1),
                                                                       1850, 2018, detrend=False)

print('CCSM4/MPI 100 vs. 200 ens  r = '+str(np.round(corr_ccsm4_100_200,3)))
print('CCSM4/MPI 100 vs. 1000 ens r = '+str(np.round(corr_ccsm4_1000_100,3)))
print('CCSM4/MPI 200 vs. 1000 ens r = '+str(np.round(corr_ccsm4_1000_200,3)))

print('CCSM4/MPI 100 vs. 200 ens  ce = '+str(np.round(ce_ccsm4_100_200,3)))
print('CCSM4/MPI 100 vs. 1000 ens ce = '+str(np.round(ce_ccsm4_1000_100,3)))
print('CCSM4/MPI 200 vs. 1000 ens ce = '+str(np.round(ce_ccsm4_1000_200,3)))